<a href="https://colab.research.google.com/github/dellanie/Project-Capstone/blob/main/Project_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up imports and preprocessing




**PART 2**

In [ ]:
# Import the following libraries
import pandas as pd
import numpy as np

# NLTK libraries
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Visualization libraries
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image # for world cloud image

# Spacy for preprocessing
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')


# To change date to datetime
from datetime import datetime
import re

from collections import Counter
import string
import scipy.sparse

# Gensim libraries
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
#import pyLDAvis.gensim
from gensim.models import CoherenceModel
from gensim import matutils

# Avoid warnings
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.5 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!python -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']):
       output = []
       for sent in texts:
             doc = nlp(sent)
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

2023-04-14 01:34:14.596207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
!pip install --upgrade gensim
from gensim import corpora, models, similarities, downloader
# Creating the object for LDA model using gensim library


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyLDAvis import gensim
LDA =models.ldamodel.LdaModel

# Build LDA model
# lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
#                 chunksize=1000, passes=50,iterations=100)

In [ ]:
import pyLDAvis.gensim_models
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

*Section* 2


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('Clusters.xlsx')

In [ ]:
len(df['Cluster 1'])

1

In [ ]:
df['Cluster 1'] = df['Cluster 1'].replace(',',' ')

In [ ]:
df['Cluster 1'].dtype

dtype('O')

In [ ]:
df['Cluster 1'] = df['Cluster 1'].astype(str)

In [ ]:
type(df['Cluster 1'])

pandas.core.series.Series

In [ ]:
len(df['Cluster 1'])

1

In [ ]:
text_list=df['Cluster 1'].tolist()
print(text_list[0])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[0])

understanding,stakeholders,people,experiences,feedback,understand,take,performative,ensure,marginalized,important,real,may,biases,policies,data,policy,help,use,work
['understanding', 'stakeholder', 'people', 'experience', 'feedback', 'performative', 'ensure', 'important', 'real', 'bias', 'policy', 'datum', 'policy', 'help', 'use', 'work']


In [ ]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [ ]:
# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50,iterations=100)
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -3.3369912862300875

Coherence Score:  1.0


In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary,mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.002373 -0.001742       1        1  95.288479
2     -0.000591  0.000438       2        1   1.177881
1     -0.000598  0.000429       3        1   1.177880
4     -0.000594  0.000434       4        1   1.177880
3     -0.000589  0.000441       5        1   1.177880, topic_info=             Term      Freq     Total Category  logprob  loglift
9          policy  1.000000  1.000000  Default  15.0000  15.0000
0            bias  1.000000  1.000000  Default  14.0000  14.0000
1           datum  1.000000  1.000000  Default  13.0000  13.0000
2          ensure  1.000000  1.000000  Default  12.0000  12.0000
3      experience  1.000000  1.000000  Default  11.0000  11.0000
..            ...       ...       ...      ...      ...      ...
11    stakeholder  0.012564  1.013168   Topic5  -2.7080   0.0515
12  understanding  0.012564  1.013168   Topic5  -2.7080   0.0515
13            use  0.012564  1.013168   Topic5  -2.7080   0.0515
14           work  0.012564  1.013168   Topic5  -2.7080   0.0515
9          policy  0.012564  1.815646   Topic5  -2.7081  -0.5319

[90 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.987003           bias
1         1  0.987003          datum
2         1  0.987003         ensure
3         1  0.987003     experience
4         1  0.987003       feedback
5         1  0.987003           help
6         1  0.987003      important
7         1  0.987003         people
8         1  0.987003   performative
9         1  1.101536         policy
10        1  0.987003           real
11        1  0.987003    stakeholder
12        1  0.987003  understanding
13        1  0.987003            use
14        1  0.987003           work, R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])

Communicate the policy to all employees and stakeholders, emphasizing the importance of DEI and the organization's commitment to creating a more inclusive workplace.

In [ ]:
df['Cluster 2'] = df['Cluster 2'].astype(str)

In [ ]:
text_list=df['Cluster 2'].tolist()
print(text_list[0])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[0])

dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50,iterations=100)
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

encourage,communication,equity,groups,organizations,hiring,job,inclusive,opportunities,team,candidates,culture,provide,training,create,help,inclusion,employees,diverse,diversity
['communication', 'equity', 'group', 'organization', 'hiring', 'job', 'inclusive', 'opportunity', 'team', 'candidate', 'culture', 'provide', 'training', 'help', 'inclusion', 'employee', 'diverse', 'diversity']

Perplexity:  -3.5518220066918267

Coherence Score:  1.0


In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary,mds='mmds')
vis

PreparedData(topic_coordinates=                  x             y  topics  cluster       Freq
topic                                                        
4     -1.820986e-15  6.998217e-15       1        1  95.784471
0      5.024384e-15 -1.807924e-14       2        1   1.053884
1     -1.566983e-15  3.342541e-15       3        1   1.053883
3     -1.474815e-15  3.834658e-15       4        1   1.053882
2     -1.616010e-16  3.903823e-15       5        1   1.053881, topic_info=         Term      Freq  Total Category  logprob  loglift
17   training  1.000000    1.0  Default  18.0000     18.0
10  inclusion  1.000000    1.0  Default  17.0000     17.0
0   candidate  1.000000    1.0  Default  16.0000     16.0
9      hiring  1.000000    1.0  Default  15.0000     15.0
5    employee  1.000000    1.0  Default  14.0000     14.0
..        ...       ...    ...      ...      ...      ...
12        job  0.010539    1.0   Topic5  -2.8904     -0.0
8        help  0.010539    1.0   Topic5  -2.8904     -0.0
7       group  0.010539    1.0   Topic5  -2.8904     -0.0
5    employee  0.010539    1.0   Topic5  -2.8904     -0.0
0   candidate  0.010539    1.0   Topic5  -2.8904     -0.0

[108 rows x 6 columns], token_table=      Topic  Freq           Term
term                            
0         1   1.0      candidate
1         1   1.0  communication
2         1   1.0        culture
3         1   1.0        diverse
4         1   1.0      diversity
5         1   1.0       employee
6         1   1.0         equity
7         1   1.0          group
8         1   1.0           help
9         1   1.0         hiring
10        1   1.0      inclusion
11        1   1.0      inclusive
12        1   1.0            job
13        1   1.0    opportunity
14        1   1.0   organization
15        1   1.0        provide
16        1   1.0           team
17        1   1.0       training, R=18, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])

There is focus on the importance of diversity, equity, and inclusion in the workplace, which could be creating an inclusive culture, fair hiring practices, and providing opportunities for all employees to succeed.

In [ ]:
df['Cluster 3'] = df['Cluster 3'].astype(str)

In [ ]:
text_list=df['Cluster 3'].tolist()
print(text_list[0])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[0])

dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50,iterations=100)
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

achieving,goal,responsibilities,achieve,towards,everyone,help,specific,clear,evaluate,identify,monitor,objectives,regularly,track,ensure,goals,progress,action,plan
['achieving', 'goal', 'responsibility', 'help', 'specific', 'clear', 'monitor', 'objective', 'track', 'ensure', 'goal', 'progress', 'action', 'plan']

Perplexity:  -3.2090685885020664

Coherence Score:  1.0


In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary,mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.002688 -0.001972       1        1  94.660109
2     -0.000670  0.000496       2        1   1.334973
3     -0.000677  0.000486       3        1   1.334973
1     -0.000673  0.000491       4        1   1.334973
4     -0.000668  0.000499       5        1   1.334972, topic_info=              Term      Freq     Total Category  logprob  loglift
4             goal  1.000000  1.000000  Default  13.0000  13.0000
0        achieving  1.000000  1.000000  Default  12.0000  12.0000
1           action  1.000000  1.000000  Default  11.0000  11.0000
2            clear  1.000000  1.000000  Default  10.0000  10.0000
3           ensure  1.000000  1.000000  Default   9.0000   9.0000
..             ...       ...       ...      ...      ...      ...
9         progress  0.014377  1.015508   Topic5  -2.5649   0.0587
10  responsibility  0.014377  1.015508   Topic5  -2.5649   0.0587
11        specific  0.014377  1.015508   Topic5  -2.5649   0.0587
12           track  0.014377  1.015508   Topic5  -2.5649   0.0587
4             goal  0.014376  1.813903   Topic5  -2.5650  -0.5214

[78 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         1  0.984729       achieving
1         1  0.984729          action
2         1  0.984729           clear
3         1  0.984729          ensure
4         1  1.102595            goal
5         1  0.984729            help
6         1  0.984729         monitor
7         1  0.984729       objective
8         1  0.984729            plan
9         1  0.984729        progress
10        1  0.984729  responsibility
11        1  0.984729        specific
12        1  0.984729           track, R=13, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 5])

 identify specific goals we should aim to achieve. For example, you may want to increase representation of underrepresented groups, reduce bias in hiring, or create a more inclusive workplace culture.

Plans or strategies to achieve desired outcomes should be put in place.



In [ ]:
df['Cluster 4'] = df['Cluster 4'].astype(str)

In [ ]:
text_list=df['Cluster 4'].tolist()
print(text_list[0])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[0])

dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50,iterations=100)
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

make,avoid,role,involves,show,others,interrupting,impact,conversations,attentive,respect,patient,empathy,additionally,impactful,perspective,enhance,listening,speakers,important
['role', 'show', 'other', 'impact', 'conversation', 'attentive', 'respect', 'patient', 'empathy', 'impactful', 'perspective', 'enhance', 'speaker', 'important']

Perplexity:  -3.340461643218994

Coherence Score:  1.0


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary,mds='mmds')
vis

PreparedData(topic_coordinates=                  x             y  topics  cluster       Freq
topic                                                        
0      5.938432e-18 -1.050517e-17       1        1  94.660066
1      5.896138e-17 -5.727002e-17       2        1   1.334985
2     -1.867676e-17  3.127639e-17       3        1   1.334983
4     -1.993138e-17  3.018005e-17       4        1   1.334983
3     -2.629168e-17  6.318745e-18       5        1   1.334983, topic_info=            Term     Freq  Total Category  logprob  loglift
0      attentive  1.00000    1.0  Default  14.0000     14.0
1   conversation  1.00000    1.0  Default  13.0000     13.0
2        empathy  1.00000    1.0  Default  12.0000     12.0
3        enhance  1.00000    1.0  Default  11.0000     11.0
4         impact  1.00000    1.0  Default  10.0000     10.0
..           ...      ...    ...      ...      ...      ...
7          other  0.01335    1.0   Topic5  -2.6391     -0.0
8        patient  0.01335    1.0   Topic5  -2.6391     -0.0
9    perspective  0.01335    1.0   Topic5  -2.6391     -0.0
10       respect  0.01335    1.0   Topic5  -2.6391     -0.0
11          role  0.01335    1.0   Topic5  -2.6391     -0.0

[84 rows x 6 columns], token_table=      Topic  Freq          Term
term                           
0         1   1.0     attentive
1         1   1.0  conversation
2         1   1.0       empathy
3         1   1.0       enhance
4         1   1.0        impact
5         1   1.0     impactful
6         1   1.0     important
7         1   1.0         other
8         1   1.0       patient
9         1   1.0   perspective
10        1   1.0       respect
11        1   1.0          role
12        1   1.0          show
13        1   1.0       speaker, R=14, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 5, 4])

The speaker/representative has a key role in creating an inclusive and respectful workplace culture, and can enhance their impact by being mindful of these qualities and striving to have impactful conversations.